# Đồ án môn học Lập trình song song ứng dụng

# Phát hiện khuôn mặt bằng Cascaded-Classifiers

## Giáo viên: Trần Trung Kiên

## Sinh viên:
- 1712628 - Huỳnh Hiếu Nhân
- 1712657 - Chung Kiệt Phong
- 1712869 - Đào Duy Tuấn

---

# 1. Mô tả ứng dụng

- Nhóm em chọn ứng dụng Phát hiện khuôn mặt bằng Cascaded-Classifiers
	- Input:
	- Output:
	- Ý nghĩa của ứng dụng trong thực tế:
- Lý do cần tăng tốc ứng dụng:
- Tiềm năng song song hóa của ứng dụng: (Nhận định sơ bộ thôi; ví dụ, ứng dụng huấn luyện Convolutional Neural Network có tiềm năng song song hóa vì trong đó có phép convolution, phép nhân ma trận, ...)


---